In [61]:
import pandas as pd
from pathlib import Path
import csv
import uuid
import requests
import json

data_dir = Path("./data")
csv_files = list(data_dir.glob("*/*.csv"))

In [12]:
# List to hold individual DataFrames
dfs = []

# Loop through each CSV file
for file in csv_files:
    # Extract city name from the parent directory
    city = file.parent.name

    if city == "Jakarta":
        interaction_id = file.stem.split("_")[-2]
    else:
        interaction_id = file.stem.split("_")[-1]

    print(city, interaction_id)
    df = pd.read_csv(file)
    df["city"] = city
    df["interaction_id"] = interaction_id
    dfs.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True).drop(columns=["sentiment", "intonation", "pitch range", "start_time", "end_time", "chunk", "city"])

filtered_df = combined_df[combined_df["speaker"] != "silence"]
filtered_df

Semarang esUREQ
Semarang Jhrupw
Semarang eJLBQp
Semarang IaLwyJ
Semarang apngOU
Semarang SlqTkJ
Semarang iyqAhC
Semarang NPeExN
Semarang zIKpCE
Semarang ZxLRNx
Semarang LcRQus
Semarang VJPhZY
Semarang XIVrke
Semarang VhVXED
Semarang dGxrgN
Semarang eoJcNt
Semarang Jhrupw
Semarang OgLHqq
Semarang DDfhKH
Semarang oAuBPR
Yogyakarta WFlnvl
Yogyakarta garokb
Yogyakarta dIVeRB
Yogyakarta myKkSw
Yogyakarta wvKPCZ
Yogyakarta ipqsZV
Yogyakarta YKovDp
Yogyakarta aDZlQA
Yogyakarta IrfIfL
Yogyakarta TuUveh
Jakarta uCrDqG
Jakarta Kzrjte
Jakarta PNywPq
Jakarta LDUBDz
Jakarta xXsVkl
Jakarta giMmXj
Jakarta  
Jakarta pSomjZ
Jakarta GGFHDy
Jakarta vWXMGx
Jakarta defbuu
Jakarta pOzHft
Jakarta CCUWeQ
Jakarta oNeJTv
Jakarta IkwvuR
Jakarta iamfTI
Jakarta fejWQm
Jakarta NXwePv
Jakarta ilTtKc
Jakarta buRfJA


,speaker,transcript,interaction_id
1,Agent,"Selamat sore dengan Naira, bagaimana saya bisa...",esUREQ
2,Customer,Sore Mbak Naira,esUREQ
3,Agent,"Sori Bu, dengan Ibu siapa saya berbicara?",esUREQ
4,Customer,Saya Sisca,esUREQ
5,Agent,"Baik Ibu Sisca, terial apa yang bisa saya bantu?",esUREQ
...,...,...,...
3594,Agent,Baik terima kasih untuk menerima masukannya Ba...,buRfJA
3595,Customer,Ya ya ya sudah lah kalau gitu lah,buRfJA
3596,Agent,Baik ada lagi bisa saya bantu Bapak Doni,buRfJA
3597,Customer,Gitu aja,buRfJA


In [28]:
grouped = filtered_df.groupby("interaction_id")

json_data = {}
grouped_data = {}

for _, row in filtered_df.iterrows():
    interaction_id = row["interaction_id"]
    if interaction_id not in grouped_data:
        grouped_data[interaction_id] = []

    grouped_data[interaction_id].append({
        "role": row["speaker"],
        "message": row["transcript"],
        "is_final": "final"
    })


with open("./result/output.json", "w", encoding="utf-8") as f:
    json.dump(grouped_data, f, ensure_ascii=False, indent=4)


Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Agent
Agent
Customer
Customer
Agent
Agent
Customer
Agent
Customer
Agent
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Customer
Customer
Agent
Customer
Customer
Agent
Customer
Agent
Customer
Customer
Agent
Agent
Customer
Agent
Agent
Customer
Agent
Customer
Agent
Customer
Customer
Customer
Agent
Agent
Agent
Agent
Customer
Customer
Agent
Agent
Agent
Agent
Customer
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Customer
Agent
Agent
Customer
Agent
Customer
Customer
Agent
Customer
Customer
Agent
Customer
Customer
Agent
Agent
Agent
Agent
Customer
Agent
Customer
Customer
Agent
Customer
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agent
Customer
Agen

{'esUREQ': [{'role': 'Agent',
   'message': 'Selamat sore dengan Naira, bagaimana saya bisa membantu?',
   'is_final': 'final'},
  {'role': 'Customer', 'message': 'Sore Mbak Naira', 'is_final': 'final'},
  {'role': 'Agent',
   'message': 'Sori Bu, dengan Ibu siapa saya berbicara?',
   'is_final': 'final'},
  {'role': 'Customer', 'message': 'Saya Sisca', 'is_final': 'final'},
  {'role': 'Agent',
   'message': 'Baik Ibu Sisca, terial apa yang bisa saya bantu?',
   'is_final': 'final'},
  {'role': 'Customer',
   'message': 'Ee Mbak, saya mau cek dong kayaknya ini ada transaksi yang enggak saya ee lakukan gitu, saya mau sanggah Mbak transaksinya, bisa enggak ya?',
   'is_final': 'final'},
  {'role': 'Agent',
   'message': 'Baik, begitu ya Bu, Ibu tidak usah khawatir di sini saya akan proses untuk pengecekan data. Untuk nomor kartu kreditnya Bu bisa disebutkan?',
   'is_final': 'final'},
  {'role': 'Customer',
   'message': 'Nomor kartunya di 5244',
   'is_final': 'final'},
  {'role': 'Agen

# HIT TO LLM TO GET SUMMARY

In [59]:
with open("./result/output.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# function to hit endpoint
def hit_endpoint(url, interaction_id, subtitle):
    auth = ('', '')
    payload = {
        "seq_id": 1,
        "interaction_id": f"{uuid.uuid4()}",
        "subtitle": subtitle
    }
    try:
        response = requests.post(
            url,
            json=payload,
            auth=auth
        )
        response.raise_for_status()
        return response.json()  # This returns just the response data

    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e} for interaction_id: {interaction_id}")

url_summary = "http://localhost:8000/rtaa/api/v1/call_summarizer_sentiment"
url_wi = "http://localhost:8000/rtaa/api/v1/call_intent_detection"

# get all list of interaction_id
for interaction_id, subtitle in data.items():
    # if interaction_id == "myKkSw":
        summary= hit_endpoint(url_summary, interaction_id, subtitle)['summarizer']
        working_instruction = hit_endpoint(url_wi, interaction_id, subtitle)

        set_wi= set()

        if working_instruction:
            for wi in working_instruction['working_instruction']:
                set_wi.add(wi['document'])
        with open(f"./result/llm_output.csv", "a", encoding="utf-8") as f:
            writer = csv.writer(f)  # Use csv writer for proper quoting and escaping

            if f.tell() == 0:  # Write header only once at the beginning of the file
                header = ["interaction_id", "transcript", "summary", "working_instruction"]
                writer.writerow(header)

            row = [interaction_id, subtitle, summary, str(set_wi)] # Convert set_wi to string
            writer.writerow(row)

Error making request: 500 Server Error: Internal Server Error for url: http://localhost:8000/rtaa/api/v1/call_intent_detection for interaction_id: myKkSw
Error making request: 500 Server Error: Internal Server Error for url: http://localhost:8000/rtaa/api/v1/call_intent_detection for interaction_id: ipqsZV
Error making request: 500 Server Error: Internal Server Error for url: http://localhost:8000/rtaa/api/v1/call_intent_detection for interaction_id: pSomjZ


# READ OUTPUT from LLM

In [115]:
llm_output = pd.read_csv("./result/llm_output.csv", index_col=0)

In [111]:
import numpy as np
from evaluate import load
cer = load("cer")

def calculate_cer(expected, result):
    if pd.isna(expected) or pd.isna(result):
        print(expected, result)
        return np.nan

    predictions = [result]
    references = [expected]
    return float(cer.compute(predictions=predictions, references=references))

# MAP expected data to llm_output

In [125]:
# Load expected nomor kartu
expected_card = pd.read_excel("./data/root_cause_2.xlsx", sheet_name="No Kartu Accuracy", index_col=1)
expected_cash = pd.read_excel("./data/root_cause_2.xlsx", sheet_name="Nominal Accuracy", index_col=1)

llm_output_1 = llm_output.merge(
    expected_card[["No kartu yang disebutkan nasabah", "Expected WI"]],
    on="interaction_id",
    how="left"
)

,transcript,summary,working_instruction,No kartu yang disebutkan nasabah,Expected WI
interaction_id,,,,,
esUREQ,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Sisca, nomor kartu kredit ber...","{'Sanggahan Transaksi Kartu Kredit', 'Blokir K...",5244260051609330,1. WI - Sanggahan transaksi kartu debit\n2. No...
Jhrupw,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Martupa, NIK 3323042301990001...","{'Pemblokiran OCTO Mobile', 'Ketentuan Pencair...",5244621812878210,1. WI - blokir octo mobile\n2. No Kartu 5244 ...
eJLBQp,"[{'role': 'Customer', 'message': 'Selamat sore...","1. Data nasabah: Martu, nomor kartu kredit 533...",set(),5336951632147900,1. WI - Penanganan Complaint Tidak Terima PIN ...
IaLwyJ,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Sisca Christiana, Nomor KTP: ...",{'Pembukaan Xtra Dana Nasabah Tidak Terundang'},3323042301990000,1. WI - pembukaan xtra dana ntb\n2. No NIK 332...
apngOU,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Bapak Martupa, nomor kartu kr...","{'Pendaftaran E-Statement', 'Pengajuan Kenaika...",5244621811243450,1. WI Pengajuan Kenaikan Limit Kartu Kredit S...
SlqTkJ,"[{'role': 'Agent', 'message': 'Halo, selamat s...","1. Data nasabah: Bapak Martupa, nomor kartu kr...",{'Pendaftaran E-Statement'},NaN,NaN
iyqAhC,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Martupa, nomor kartu kredit 5...",set(),5244654310008210,1. WI - Close Card\n2. No Kartu 5244 6543 1000...
NPeExN,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Afaya, nomor kartu ATM 5336 6...",{'Pendaftaran E-Statement'},NaN,NaN
zIKpCE,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Intan, nomor kartu debit 5336...","{'Pengaktifan Kartu Debit Karena Terblokir', '...",5336400012242000,1. WI - blokir kartu debit\n2. No Kartu 5336 ...


In [127]:
llm_output_2 = llm_output_1.merge(
    expected_cash[["Nominal yang disebutkan nasabah"]],
    on="interaction_id",
    how="left"
)

In [128]:
llm_output_2

,transcript,summary,working_instruction,No kartu yang disebutkan nasabah,Expected WI,Nominal yang disebutkan nasabah
interaction_id,,,,,,
esUREQ,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Sisca, nomor kartu kredit ber...","{'Sanggahan Transaksi Kartu Kredit', 'Blokir K...",5244260051609330,1. WI - Sanggahan transaksi kartu debit\n2. No...,24.500.750
Jhrupw,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Martupa, NIK 3323042301990001...","{'Pemblokiran OCTO Mobile', 'Ketentuan Pencair...",5244621812878210,1. WI - blokir octo mobile\n2. No Kartu 5244 ...,"14,500,000\n"
eJLBQp,"[{'role': 'Customer', 'message': 'Selamat sore...","1. Data nasabah: Martu, nomor kartu kredit 533...",set(),5336951632147900,1. WI - Penanganan Complaint Tidak Terima PIN ...,4.356.000
IaLwyJ,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Sisca Christiana, Nomor KTP: ...",{'Pembukaan Xtra Dana Nasabah Tidak Terundang'},3323042301990000,1. WI - pembukaan xtra dana ntb\n2. No NIK 332...,11.567.550
apngOU,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Bapak Martupa, nomor kartu kr...","{'Pendaftaran E-Statement', 'Pengajuan Kenaika...",5244621811243450,1. WI Pengajuan Kenaikan Limit Kartu Kredit S...,35.500.650
SlqTkJ,"[{'role': 'Agent', 'message': 'Halo, selamat s...","1. Data nasabah: Bapak Martupa, nomor kartu kr...",{'Pendaftaran E-Statement'},NaN,NaN,NaN
iyqAhC,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Martupa, nomor kartu kredit 5...",set(),5244654310008210,1. WI - Close Card\n2. No Kartu 5244 6543 1000...,1.735.500
NPeExN,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Afaya, nomor kartu ATM 5336 6...",{'Pendaftaran E-Statement'},NaN,NaN,NaN
zIKpCE,"[{'role': 'Agent', 'message': 'Selamat sore de...","1. Data nasabah: Intan, nomor kartu debit 5336...","{'Pengaktifan Kartu Debit Karena Terblokir', '...",5336400012242000,1. WI - blokir kartu debit\n2. No Kartu 5336 ...,3.467.750


In [129]:
# write to excell
llm_output_2.to_excel("./result/llm_output.xlsx")